In [1]:
import seaborn as sns
sns.set_palette('husl')
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import hopsworks
import pandas as pd

In [3]:
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/206998
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
weather_df = pd.read_csv("https://raw.githubusercontent.com/Antares28/Beijing-Weather-Prediction/main/data/Beijing_Weather.csv")
weather_df.columns = ["date", "quality_level", "aqi", "aqi_rank_in_china", "pm25", "pm10", "so2", "no2", "co", "o3", "weather_condition", "temperature", "wind_force_and_wind_direction"]
weather_df

,date,quality_level,aqi,aqi_rank_in_china,pm25,pm10,so2,no2,co,o3,weather_condition,temperature,wind_force_and_wind_direction
0,2013-10-28,严重污染,306,106,255,277,30,105,2.60,15,霾/阵雨,7℃/16℃,无持续风向≤3级/无持续风向≤3级
1,2013-10-29,良,62,22,39,62,10,46,0.91,27,晴/晴,2℃/16℃,无持续风向≤3级/无持续风向≤3级
2,2013-10-30,良,99,61,71,101,11,72,1.18,14,晴/晴,4℃/17℃,无持续风向≤3级/无持续风向≤3级
3,2013-10-31,中度污染,176,98,135,162,10,96,1.62,2,多云/霾,5℃/14℃,无持续风向≤3级/无持续风向≤3级
4,2013-11-01,重度污染,231,102,181,202,14,100,1.89,0,霾/阴,5℃/12℃,无持续风向≤3级/无持续风向≤3级
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3207,2024-01-03,优,31,46,16,33,2,32,0.38,38,晴/晴,-7℃/5℃,南风1-3级/北风1-3级
3208,2024-01-04,良,65,162,45,71,2,44,0.66,28,晴/晴,-5℃/7℃,西北风1-3级/西北风1-3级
3209,2024-01-05,优,41,57,19,44,1,32,0.42,36,晴/晴,-4℃/5℃,西南风1-3级/西北风1-3级
3210,2024-01-06,优,18,3,6,14,2,11,0.23,47,晴/晴,-10℃/3℃,西北风1-3级/北风1-3级


In [5]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3212 entries, 0 to 3211
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   date                           3212 non-null   object 
 1   quality_level                  3212 non-null   object 
 2   aqi                            3212 non-null   int64  
 3   aqi_rank_in_china              3212 non-null   int64  
 4   pm25                           3212 non-null   int64  
 5   pm10                           3212 non-null   int64  
 6   so2                            3212 non-null   int64  
 7   no2                            3212 non-null   int64  
 8   co                             3212 non-null   float64
 9   o3                             3212 non-null   int64  
 10  weather_condition              3212 non-null   object 
 11  temperature                    3212 non-null   object 
 12  wind_force_and_wind_direction  3212 non-null   o

In [6]:
weather_df.describe().T

,count,mean,std,min,25%,50%,75%,max
aqi,3212.0,86.164384,61.654080,11.00,44.00,69.00,108.0000,478.00
aqi_rank_in_china,3212.0,194.086550,119.703204,0.00,85.00,190.00,312.0000,367.00
pm25,3212.0,54.023350,53.149294,2.00,18.00,37.00,72.0000,476.00
pm10,3212.0,81.796389,71.795691,0.00,37.00,64.00,106.0000,1563.00
so2,3212.0,7.545143,12.091503,1.00,2.00,3.00,7.0000,133.00
no2,3212.0,37.414072,21.601017,1.00,22.00,33.00,48.0000,145.00
co,3212.0,0.852590,0.715210,0.07,0.44,0.66,0.9925,8.11
o3,3212.0,60.390411,37.057786,0.00,33.00,55.00,83.0000,200.00


In [7]:
# Drop columns that do not have predictive power
weather_df.drop('quality_level', axis=1, inplace=True)
weather_df.drop('aqi_rank_in_china', axis=1, inplace=True)
weather_df

,date,aqi,pm25,pm10,so2,no2,co,o3,weather_condition,temperature,wind_force_and_wind_direction
0,2013-10-28,306,255,277,30,105,2.60,15,霾/阵雨,7℃/16℃,无持续风向≤3级/无持续风向≤3级
1,2013-10-29,62,39,62,10,46,0.91,27,晴/晴,2℃/16℃,无持续风向≤3级/无持续风向≤3级
2,2013-10-30,99,71,101,11,72,1.18,14,晴/晴,4℃/17℃,无持续风向≤3级/无持续风向≤3级
3,2013-10-31,176,135,162,10,96,1.62,2,多云/霾,5℃/14℃,无持续风向≤3级/无持续风向≤3级
4,2013-11-01,231,181,202,14,100,1.89,0,霾/阴,5℃/12℃,无持续风向≤3级/无持续风向≤3级
...,...,...,...,...,...,...,...,...,...,...,...
3207,2024-01-03,31,16,33,2,32,0.38,38,晴/晴,-7℃/5℃,南风1-3级/北风1-3级
3208,2024-01-04,65,45,71,2,44,0.66,28,晴/晴,-5℃/7℃,西北风1-3级/西北风1-3级
3209,2024-01-05,41,19,44,1,32,0.42,36,晴/晴,-4℃/5℃,西南风1-3级/西北风1-3级
3210,2024-01-06,18,6,14,2,11,0.23,47,晴/晴,-10℃/3℃,西北风1-3级/北风1-3级


In [8]:
#Write the features to the feature store as a Feature Group
weather_fg = fs.get_or_create_feature_group(
    name="weather",
    version=1,
    primary_key=["date", "aqi", "pm25", "pm10", "so2", "no2", "co", "o3", "weather_condition", "temperature", "wind_force_and_wind_direction"],
    description="Beijing Weather dataset")
weather_fg.insert(weather_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/206998/fs/206917/fg/407921


Uploading Dataframe: 0.00% |          | Rows 0/3212 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/206998/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x26c9dc71c70>, None)